In [1]:
import pandas as pd
import json

In [2]:
! ls -l

total 38096
-rw-r--r--  1 aaaa  staff     10784 May 27 14:23 bitcoin_tx_loader.ipynb
drwxr-xr-x  2 aaaa  staff        64 May 27 14:23 bitcoincli
-rw-r--r--  1 aaaa  staff     12481 May 27 14:23 blockstream_bitcoin_loader.ipynb
-rw-r--r--  1 aaaa  staff   9120248 May 27 14:23 blockstream_block_834577_txs.csv
-rw-r--r--  1 aaaa  staff       258 May 27 14:23 main.py
-rw-r--r--  1 aaaa  staff  10347949 May 27 14:23 rpc_block_834577_txs.csv
-rw-r--r--  1 aaaa  staff         0 May 27 14:23 tmp.ipynb


In [111]:
d = pd.read_csv('./rpc_block_834577_txs.csv')#['hex'].map(lambda x: float(x))

def decode_v(x):
    r = x[1:-1]
    r = r.replace("'",'"')
    s = r.split(': ')[0]+': '
    r = r.split(s)[1:]
    r = [s+i for i in r]
    r = [i[:-2] if i[-2:]==', ' else i for i in r]
    r = [json.loads(i) for i in r]
    return r

d['vout'] = d['vout'].map(decode_v)
d['vin'] = d['vin'].map(decode_v)

In [161]:
t = enumerate(d['vout'].values)
t = [{**{'index':i},**j} for (i,v) in t for j in v]
t = pd.DataFrame.from_dict(t)
vout = t['scriptPubKey'].values
vout = pd.DataFrame.from_records(vout)
vout = pd.concat([t.iloc[:,:2], vout], axis=1)
vout = pd.merge(vout,d.reset_index(), how='left', on='index')

In [160]:
t = enumerate(d['vin'].values)
t = [{**{'index':i},**j} for (i,v) in t for j in v]
t = pd.DataFrame.from_dict(t)

vin = t['scriptSig'].map(lambda x: x if x==x else {'asm': None, 'hex': None})#.values
vin = pd.DataFrame.from_records(vin)
vin = pd.concat([t.iloc[:,:2], vin], axis=1)
vin = pd.merge(vin,d.reset_index(), how='left', on='index')

In [166]:
cols = ['index', 'value','time','address', 'txid', 'hash', 'asm', 'type',
        'version','vin', 'vout', 'hex_y', 'blockhash']
vout[cols].head()

,index,value,time,address,txid,hash,asm,type,version,vin,vout,hex_y,blockhash
0,0,6.391820,1710369642,bc1qxhmdufsvnuaaaer4ynz88fspdsxq2h9e9cetdj,f6537dcf21009398fb581fb2a09daf05699410e6692ed4...,1f0c7bc9b08d668f32d55eaedb4e6ab47bb60d71185e6e...,0 35f6de260c9f3bdee47524c473a6016c0c055cb9,witness_v0_keyhash,2,[{'coinbase': '0311bc0c046b2bf2652f466f756e647...,"[{'value': 6.39182026, 'n': 0, 'scriptPubKey':...",0200000000010100000000000000000000000000000000...,00000000000000000002b3eb358ccf34216f19688ccbb3...
1,0,0.000000,1710369642,NaN,f6537dcf21009398fb581fb2a09daf05699410e6692ed4...,1f0c7bc9b08d668f32d55eaedb4e6ab47bb60d71185e6e...,OP_RETURN aa21a9ed4372f328ae5d8710fedf6bff0d04...,nulldata,2,[{'coinbase': '0311bc0c046b2bf2652f466f756e647...,"[{'value': 6.39182026, 'n': 0, 'scriptPubKey':...",0200000000010100000000000000000000000000000000...,00000000000000000002b3eb358ccf34216f19688ccbb3...
2,1,4.723600,1710369642,3N1cBEBBqwsPFay1in2vKEyV5FzPqWJD3x,53f50b9f3ef7a020bf7ae705aaad31b39f5098c5a8a883...,a196952ad23f5301383686bf6e9e34ecc7242ca47ca4ad...,OP_HASH160 dee69899f930e03cea6e169ce367149c01b...,scripthash,2,[{'txid': '88888888899261c542fd8aed4f8f6eaf577...,"[{'value': 4.7236, 'n': 0, 'scriptPubKey': {'a...",0200000000010aca8079795581f071768145cd3f1a7c57...,00000000000000000002b3eb358ccf34216f19688ccbb3...
3,1,429.622151,1710369642,1Kr6QSydW9bFQG1mXiPNNu6WpJGmUa9i1g,53f50b9f3ef7a020bf7ae705aaad31b39f5098c5a8a883...,a196952ad23f5301383686bf6e9e34ecc7242ca47ca4ad...,OP_DUP OP_HASH160 cebb2851a9c7cfe2582c12ecaf7f...,pubkeyhash,2,[{'txid': '88888888899261c542fd8aed4f8f6eaf577...,"[{'value': 4.7236, 'n': 0, 'scriptPubKey': {'a...",0200000000010aca8079795581f071768145cd3f1a7c57...,00000000000000000002b3eb358ccf34216f19688ccbb3...
4,2,0.008688,1710369642,bc1qvdyaxax35erm30km46cmzwwq59kqj4lgrp7xst,ec5871ae2bbd1311149f92c95f608add5a7c7e617d0399...,d070a51b1f049d3ea2206edbc153a04b67a9f3b05ab61b...,0 6349d374d1a647b8bedbaeb1b139c0a16c0957e8,witness_v0_keyhash,2,[{'txid': '97ef5f573302af7b4e75f895c8e7d62b859...,"[{'value': 0.008688, 'n': 0, 'scriptPubKey': {...",02000000000101c82f112b05dcd88db52d652dc4799085...,00000000000000000002b3eb358ccf34216f19688ccbb3...
